# Part 2: Implementing the general CHSH problem via optimization problem

A continuación, implementaremos la desigualdad general de CHSH con el siguiente observable:

$$ S = U_0 \otimes V_0 + U_0 \otimes V_1 + U_1 \otimes V_0 - U_1 \otimes V_1$$

donde consideraremos el set de operadores ${U_0, U_1, V_0, V_1}$ que $\{ U \in \mathbb{U}(2) : \text{Spec}(U) = \{ \pm 1 \} \}$

Para ello, consideremos que de manera general (salvo una fase global) las matrices unitarias se pueden expresar de la siguiente manera:

$$
U(\theta, \phi, \lambda) = \begin{pmatrix}
\cos(\frac{\theta}{2}) & -e^{i \lambda}\sin(\frac{\theta}{2}) \\
e^{i \phi} \sin(\frac{\theta}{2}) & e^{i(\phi + \lambda)} \cos(\frac{\theta}{2})
\end{pmatrix}
$$

By imponing that the spectra of these $U$ matrices must be eigenvalues of $\pm 1$:

For 2x2 matrices, we've now that the eigenvalues are given by:

$$
\xi_\pm = \frac{\text{Tr}(U)}{2} \pm \sqrt{ (\frac{\text{Tr}(U)}{2})^2 - \det(U) }
$$
If we replace $ \frac{\text{Tr}(U)}{2} = \frac{1}{2} \cos(\theta/2)(1+e^{i (\phi + \lambda)}) $ and $\det{U} = e^{i(\phi + \lambda)}$

We obtain an explicit expression for the eigenvalues $\xi_pm$

$$
\xi_{\pm} = \frac{1}{2}\cos(\theta/2)(1+e^{i (\phi + \lambda)}) \pm \sqrt{\frac{1}{2}e^{i(\phi + \lambda)}(\cos^2(\theta/2))(1 + \cos(\phi + \lambda) - 2)}
$$

By inspection, one way to impose the condition that $\xi_\pm = \pm 1$ is by the following two constraint:

$$
\left\{ 
    \begin{matrix}
    \frac{1}{2}\cos(\theta/2)(1+e^{i (\phi + \lambda)}) = 0 \\
    \frac{1}{2}e^{i (\phi + \lambda)}(\cos^2(\theta/2))(1 + \cos(\phi + \lambda) - 2) = 1
    \end{matrix}
\right.
$$


By setting $x = \phi + \lambda$, we need to solve for the following system of equations:


$$
\left\{ 
    \begin{matrix}
    \frac{1}{2}\cos(\theta/2)(1+e^{i x}) = 0 & (\text{Traceless condition })\\
    \frac{1}{2}e^{i x}(\cos^2(\theta/2))(1 + \cos(x) - 2) = 1
    \end{matrix}
\right.
$$

From the first equation, we can notice that $x = \pi$, we obtain that



$$
\det{U - \xi \mathbb{I}} = \det \begin{pmatrix}
\cos(\frac{\theta}{2}) - \xi & -e^{i \lambda}\sin(\frac{\theta}{2}) \\
e^{i \phi} \sin(\frac{\theta}{2}) & e^{i(\phi + \lambda)} \cos(\frac{\theta}{2}) - \xi
\end{pmatrix}

= 

$$



$$ U(\theta, \lambda) = \begin{pmatrix}
\cos(\theta / 2) & -e^{i \lambda} \sin(\theta / 2) \\
-e^{-i \lambda} \sin(\theta / 2) & -\cos(\theta / 2) 
\end{pmatrix} \\ = \cos(\theta / 2) Z  -\sin(\theta / 2) \cos(\lambda) X -\sin(\theta / 2) \sin(\lambda) Y $$



In [1]:
import pennylane as qml
import pennylane.numpy as pnp



num_qubits = 2
num_operators = 2 * num_qubits


def ansatz(ansatz_weight, num_qubits):

    qml.RY(phi= ansatz_weight[0], wires= 0)
    qml.RY(phi= ansatz_weight[1], wires= 1)

    qml.RZ(phi= ansatz_weight[2], wires= 0)
    qml.RZ(phi= ansatz_weight[3], wires= 1)

    qml.cx(0, 1)

    qml.RY(phi= ansatz_weight[4], wires= 0)
    qml.RY(phi= ansatz_weight[5], wires= 1)

    qml.RZ(phi= ansatz_weight[6], wires= 0)
    qml.RZ(phi= ansatz_weight[7], wires= 1)


    """
    qml.BasicEntanglerLayers(weights= ansatz_weight,
                             wires= range(num_qubits),
                              )
    """
    
    pass





In [2]:
import sympy as sp
from functools import cache


# Define symbolic observables a_n and a_n'
def a_n(n):
    return sp.Symbol(f'a{n}', commutative= False)

def a_n_prime(n):
    return sp.Symbol(f"a{n}'", commutative= False)

# Function to create the sympy expression
def tuples_to_expression(term_tuples):
    expr = 0
    for coef, *variables in term_tuples:
        product = coef
        for var in variables:
            product *= var
        expr += product
    return expr

def Mn_prime_generator(M_n):
    Mprime = []
    
    if M_n.args == ():
        return a_n_prime(1)

    for term in M_n.args:
        newterm = list(term.args)

        
        for i, algebraic_terms in enumerate(newterm[1:]):

            alg_str = str(algebraic_terms)

            if "'" in alg_str:
                newterm[i + 1] = sp.Symbol(alg_str[:-1], commutative= False)

            else:
                newterm[i + 1] = sp.Symbol(alg_str + "'", commutative= False)
        
        newterm = tuple(newterm)

        Mprime.append(newterm)
    
    return tuples_to_expression(Mprime)


# Define a recursive function for the Mermin polynomial M_n
@cache
def Mermin_polynomial(n):
    if n == 1:
        # Base case: M1 = a1
        return a_n(1)
    else:
        # Recursive case: construct M_n using M_{n-1} and M'_{n-1}
        M_n_minus_1 = Mermin_polynomial(n - 1)
        
        # Create M'_n-1 by substituting each a_k with a_k'
        M_prime_n_minus_1 = Mn_prime_generator(M_n_minus_1)

        # Construct M_n according to the recursive formula
        term1 = sp.Rational(1, 2) * M_n_minus_1 * (a_n(n) + a_n_prime(n))
        term2 = sp.Rational(1, 2) * M_prime_n_minus_1 * (a_n(n) - a_n_prime(n))
        M_n = term1 + term2
        
        # Expand, simplify, and collect like terms
        M_n_expanded = sp.expand(sp.simplify(M_n))
    
        
        return M_n_expanded

# Test the function for M_2 and M_3 and simplify them
M2 = Mermin_polynomial(2)
M3 = Mermin_polynomial(3)

# Print the simplified and collected forms of M2 and M3
print("Simplified M2:", M2)
print("Simplified M3:", M3)

Simplified M2: a1*a2/2 + a1*a2'/2 + a1'*a2/2 - a1'*a2'/2
Simplified M3: a1*a2*a3'/2 + a1*a2'*a3/2 + a1'*a2*a3/2 - a1'*a2'*a3'/2


In [3]:
# Definir una función que separa los factores y los convierte en cadenas

def separar_factores(expr):
    if isinstance(expr, sp.Mul):  # Si es un producto
        factores = expr.args
    else:  # Si es un solo término
        factores = (expr,)
    return [str(factor) for factor in factores]





def generate_mermin_observable(N_qubits, weigths):
    
    #generamos el polinomio de Mermin
    Mn = Mermin_polynomial(N_qubits)

    #generamos las instrucciones para el observable
    coef_list = []

    for term in list(Mn.args):
        coef, simbolo = term.as_coeff_Mul()
        split_instructions = separar_factores(simbolo)
        split_instructions.insert(0, coef)
        
        coef_list.append(split_instructions)


    #generamos el diccionario que aplicará nuestros 
    #distintos terminos del observable.

    op_dict = {}
    for n in range(N_qubits):
        t1 = weigths[n]
        d1 = weigths[ N_qubits + n]
        
        t2 = weigths[ 2* N_qubits + n]
        d2 = weigths[ 3* N_qubits + n]

        op_dict[f"a{n + 1}"] = qml.U3(theta= t1, phi= pnp.pi - d1, delta= d1, wires= n, id=f"a{n + 1}")
        op_dict[f"a{n + 1}'"] = qml.U3(theta= t2, phi= pnp.pi - d2, delta= d2, wires= n, id= f"a{n + 1}'")


    observables = []
    for term in coef_list:
        
        #reconsider this, because optimizing for c / 2^n is the same as optimizing c ??
        coef = float(term[0])
        
        instructions = term[1:]
        
        op  = coef * op_dict[instructions[0]]

        for ai in instructions[1:]:
            nterm = op_dict[ai]
            op =  op @ nterm

        observables.append(op)
    
    pass




vec = pnp.array(pnp.random.uniform(0, 2* pnp.pi, size= 8), requires_grad= True)
generate_mermin_observable(2, vec)








In [4]:
def generate_mermin_observable(N_qubits, weigths):
    # Generamos el polinomio de Mermin
    Mn = Mermin_polynomial(N_qubits)

    # Generamos las instrucciones para el observable
    coef_list = []

    for term in list(Mn.args):
        coef, simbolo = term.as_coeff_Mul()
        split_instructions = separar_factores(simbolo)
        split_instructions.insert(0, coef)
        
        coef_list.append(split_instructions)

    # Generamos el diccionario que aplicará nuestros 
    # distintos términos del observable.
    op_dict = {}
    for n in range(N_qubits):
        t1 = weigths[n]
        d1 = weigths[ N_qubits + n]
        
        t2 = weigths[ 2* N_qubits + n]
        d2 = weigths[ 3* N_qubits + n]

        op_dict[f"a{n + 1}"] = qml.U3(theta= t1, phi= pnp.pi - d1, delta= d1, wires= n, id=f"a{n + 1}")
        op_dict[f"a{n + 1}'"] = qml.U3(theta= t2, phi= pnp.pi - d2, delta= d2, wires= n, id= f"a{n + 1}'")

    observables = []
    for term in coef_list:
        coef = float(term[0])  # Obtener el coeficiente
        instructions = term[1:]  # Instrucciones para los operadores

        # Inicializar el operador con el primer término
        op = coef * op_dict[instructions[0]]

        for ai in instructions[1:]:
            nterm = op_dict[ai]
            op = op @ nterm  # Combinamos los operadores

        observables.append(op)  # Añadimos el operador resultante a la lista

    return observables  # Devolvemos la lista de observables


In [5]:
import pennylane as qml
from pennylane import numpy as pnp

# Dispositivo cuántico
dev = qml.device('default.qubit', wires=2)

# Define el ansatz
def ansatz(params, N_qubits):
    for i in range(N_qubits):
        qml.RY(params[i], wires=i)
        qml.RZ(params[N_qubits + i], wires=i)
    qml.CNOT(wires=[0, 1])

@qml.qnode(dev)
def circuit(params):
    N_qubits = 2
    num_op_param = 4 * N_qubits
    num_ansatz_param = 2 * N_qubits

    # Separar los parámetros
    x_operator = params[:num_op_param]
    x_ansatz = params[num_op_param:]

    # Aplicar el ansatz
    ansatz(x_ansatz, N_qubits)


    # Aplicar observables asociados directamente al circuito
    mermin_obs = generate_mermin_observable(2, x_operator)

    ################################################

    # Generamos el polinomio de Mermin
    Mn = Mermin_polynomial(N_qubits)

    # Generamos las instrucciones para el observable
    coef_list = []

    for term in list(Mn.args):
        coef, simbolo = term.as_coeff_Mul()
        split_instructions = separar_factores(simbolo)
        split_instructions.insert(0, coef)
        
        coef_list.append(split_instructions)

    # Generamos el diccionario que aplicará nuestros 
    # distintos términos del observable.
    
    op_dict = {}
    for n in range(N_qubits):
        t1 = x_operator[n]
        d1 = x_operator[ N_qubits + n]
        
        t2 = x_operator[ 2* N_qubits + n]
        d2 = x_operator[ 3* N_qubits + n]

        """
        op_dict[f"a{n + 1}"] = qml.U3(theta= t1, phi= pnp.pi - d1, delta= d1, wires= n, id=f"a{n + 1}")
        op_dict[f"a{n + 1}'"] = qml.U3(theta= t2, phi= pnp.pi - d2, delta= d2, wires= n, id= f"a{n + 1}'")

        """

        print(n)
        op_dict[f"a{n + 1}"] = {'theta' : t1, 'phi' : pnp.pi - d1,
                         'delta' : d1, 
                         'wires' : n , 'id' : f"a{n + 1}"}

        op_dict[f"a{n + 1}'"] = {'theta' : t2, 'phi' : pnp.pi - d2,
                                'delta' : d2, 
                                'wires' : n , 'id' : f"a{n + 1}'"}

    

    observables = []
    coeffs = []
    for term in coef_list:
        coeff = float(term[0])  # Obtener el coeficiente
        instructions = term[1:]  # Instrucciones para los operadores

        # Inicializar el operador con el primer término

        a1 = instructions[0]
        kwargs = op_dict[a1]
        op = coeff * qml.U3(*kwargs)

        for ai in instructions[1:]:
    
            kwargs = op_dict[ai]
            op = coeff * qml.U3(*kwargs)
        

        observables.append(op)  # Añadimos el operador resultante a la lista
        coeffs.append(coeff)
    
    ham = -1 * qml.Hamiltonian(coeffs, observables)

    return qml.expval(ham)  # Devolvemos la lista de observables



theta = pnp.array( pnp.random.uniform(0, 2* pnp.pi, size= 12), requires_grad= True)
angle = [theta]
cost = [circuit(theta)]

opt = qml.GradientDescentOptimizer()
max_iter= 100
conv_tol = 1e-6

for n in range(max_iter):
    theta, prev_cost = opt.step_and_cost(circuit, theta)
    cost.append(theta)

    conv = pnp.abs(cost[-1] - prev_cost)
    if n % 10 == 0:
        print(f"Step = {n},  Cost function = {cost[-1]:.8f} ")
    if conv <= conv_tol:
        break

print("\n" f"Final value of the cost function = {cost[-1]:.8f} ")
print("\n" f"Optimal value of the first circuit parameter =    "  + str(angle[-1][0]))
print("\n" f"Optimal value of the second circuit parameter =    "  + str(angle[-1][1]))

0
1


WireError: Cannot run circuit(s) on default.qubit as they contain wires not found on the device: {'wires'}

UsageError: Line magic function `%git` not found.


In [43]:
import pennylane as qml
from pennylane import numpy as pnp

# Dispositivo cuántico
dev = qml.device("lightning.qubit", wires= 2)


# Define el ansatz
def ansatz(params, N_qubits):

    qml.RX(params[0], wires= 0)
    qml.CNOT(0, 1)
    
    """

    for j in range(4):
        for i in range(N_qubits):
            qml.RY(params[i], wires=i)
            qml.RZ(params[N_qubits + i], wires=i)
        qml.CNOT(wires=[0, 1])
    """

@qml.qnode(dev)
def circuit(params):
    N_qubits = 2  # Asegúrate de que N_qubits no sea mayor que el número de cables disponibles
    num_op_param = 4 * N_qubits
    num_ansatz_param = 2 * N_qubits

    # Separar los parámetros
    x_operator = params[:num_op_param]
    x_ansatz = params[num_op_param:]

    # Aplicar el ansatz
    ansatz(x_ansatz, N_qubits)

    # Aplicar observables asociados directamente al circuito
    #mermin_obs = generate_mermin_observable(2, x_operator)

    # Generamos el polinomio de Mermin
    Mn = Mermin_polynomial(N_qubits)

    # Generamos las instrucciones para el observable
    coef_list = []

    for term in list(Mn.args):
        coef, simbolo = term.as_coeff_Mul()
        split_instructions = separar_factores(simbolo)
        split_instructions.insert(0, coef)
        
        coef_list.append(split_instructions)

    # Generamos el diccionario que aplicará nuestros 
    # distintos términos del observable.
    
    op_dict = {}
    for n in range(N_qubits):
        t1 = x_operator[n]
        d1 = x_operator[ N_qubits + n]
        
        t2 = x_operator[ 2* N_qubits + n]
        d2 = x_operator[ 3* N_qubits + n]

        # Asignar los operadores a los cables correctos (wires)
        op_dict[f"a{n + 1}"] = {'theta' : t1, 'phi' : pnp.pi - d1,
                                 'delta' : d1, 
                                 'wires' : n , 'id' : f"a{n + 1}"}

        op_dict[f"a{n + 1}'"] = {'theta' : t2, 'phi' : pnp.pi - d2,
                                 'delta' : d2, 
                                 'wires' : n , 'id' : f"a{n + 1}'"}
        

    # Define your Hamiltonian 
    observables = []
    coeffs = []
    for term in coef_list:
        coeff = float(term[0])  # Obtener el coeficiente
        instructions = term[1:]  # Instrucciones para los operadores

        # Inicializar el operador con el primer término
        a1 = instructions[0]
        kwargs = op_dict[a1]
        op = coeff * qml.U3(**kwargs)

        for ai in instructions[1:]:
            kwargs = op_dict[ai]
            op = op @ qml.U3(**kwargs)



        observables.append(op)  # Añadimos el operador resultante a la lista
        coeffs.append(coeff)

    ham =  qml.Hamiltonian(coeffs, observables,)

    #medir en la base computacional 

    return qml.expval(ham)  # Devolvemos la lista de observables




def cost_fn(params):
    return circuit(params)

# Configuración de la optimización
N_qubits = 2
num_op_param = 4 * N_qubits
num_ansatz_param = 1
total_params = num_op_param + num_ansatz_param

# Inicializar parámetros
params = pnp.array(pnp.random.uniform(0, 2 * pnp.pi, size= total_params), requires_grad=True)

# Optimizador
opt = qml.AdagradOptimizer()
steps = 200

# Proceso de optimización
for step in range(steps):
    params = opt.step(cost_fn, params)
    if step % 10 == 0:
        cost = cost_fn(params)
        print(f"Step {step} | Cost: {cost:.4f}")

# Resultados finales
print("Optimized parameters:", params)

TypeError: 'int' object is not subscriptable

In [38]:
print( cost)

-0.0777210675872672
